In [4]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


C:\Users\Zheryu\Anaconda3\envs\image-processing\lib\site-packages\IPython\core\magics\pylab.py:161: UserWarning: pylab import has clobbered these variables: ['ifft']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


#### Import

In [1]:
import numpy as np
import pyfftw.interfaces.scipy_fftpack as spfft
import scipy.signal as spsignal
import skimage.transform as sktransform
import matplotlib.pyplot as plt

In [2]:
from filter_tools import *
from util import *
from pyramid_tools import *

In [19]:
def phase_amplify(video, 
                  magnification_factor, 
                  fl, fh, fs, 
                  attenuate_other_frequencies=False, 
                  pyramid_type="octave", 
                  sigma=0, 
                  temporal_filter=fir_window_bp
                 ):
    num_frames, h, w, num_channels = video.shape
    pyr_height = max_scf_pyr_height((h, w))

    if pyr_type is "octave":
        print("Using vanilla octave pyramid")
        filters = get_filters((h, w), 2**np.array(list(range(0,-pyr_height-1,-1)), dtype=float), 4)
    elif pyr_type is "halfOctave":
        print("Using half octave pyramid")
        filters = get_filters((h, w), 2**np.array(list(range(0,-pyr_height-1,-1)), dtype=float), 8, t_width=0.75)
    elif pyr_type is "smoothHalfOctave":
        print("Using smooth half octave pyramid.")
        filters = get_filters_smooth_window((h, w), 8, filters_per_octave=2)
    elif pyr_type is "quarterOctave":
        print("Using quarter octave pyramid.")
        filters = get_filters_smooth_window((h, w), 8, filters_per_octave=4)
    else:
        print("Invalid filter type. Specify ocatave, halfOcatave, smoothHalfOctave, or quarterOctave")
        return None
    print('Number of filters:, ', len(filters))

    yiq_video = np.zeros((num_frames, h, w, num_channels))
    fft_video = np.zeros((num_frames, h, w), dtype=complex64)

    for i in range(num_frames):
        yiq_video[i] = rgb2yiq(video[i])
        fft_video[i] = spfft.fftshift(spfft.fft2(yiq_video[i][:,:,0]))

    magnified_y_channel = np.zeros((num_frames, h, w), dtype=complex64)
    dc_frame_index = 0
    for i in range(1,len(filters)-1):
        print("processing level "+str(i))

        dc_frame = spfft.ifft2(spfft.ifftshift(filters[i]*fft_video[dc_frame_index]))    
        dc_frame_no_mag = dc_frame / np.abs(dc_frame)    
        dc_frame_phase = np.angle(dc_frame)

        total = np.zeros(fft_video.shape, dtype=float)
        filtered = np.zeros(fft_video.shape, dtype=complex64)

        for j in range(num_frames):
            filtered[j] = spfft.ifft2(spfft.ifftshift(filters[i]*fft_video[j]))
            total[j] = simplify_phase(np.angle(filtered[j]) - dc_frame_phase)

        print("bandpassing...")
        total = temporal_filter(total, fl/fs, fh/fs).astype(float)

        for j in range(num_frames):
            phase_of_frame = total[j]
            if sigma != 0:
                phase_of_frame = amplitude_weighted_blur(phase_of_frame, np.abs(filtered[j]), sigma)

            phase_of_frame *= magnification_factor

            if attenuate_other_frequencies:
                temp_orig = np.abs(filtered[j])*dc_frame_no_mag
            else:
                temp_orig = filtered[j]
            magnified_component = 2*filters[i]*spfft.fftshift(spfft.fft2(temp_orig*np.exp(1j*phase_of_frame)))

            magnified_y_channel[j] = magnified_y_channel[j] + magnified_component

    for i in range(num_frames):
            magnified_y_channel[i] = magnified_y_channel[i] + (fft_video[i]*(filters[-1]**2))

    out = np.zeros(yiq_video.shape[:-1])
    print(out.shape)

    for i in range(num_frames):
        out_frame  = np.real(spfft.ifft2(spfft.ifftshift(magnified_y_channel[i])))
        # out_frame  = np.dstack((np.real(spfft.ifft2(spfft.ifftshift(magnified_y_channel[i]))), yiq_video[i,:,:,1:3]))
        out[i] = out_frame
        # out[i] = yiq2rgb(out_frame)

    return out.clip(min=0, max=1)


<>:12: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:15: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:18: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:21: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:12: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:15: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:18: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:21: SyntaxWarning: "is" with a literal. Did you mean "=="?
/tmp/ipykernel_63481/614351666.py:12: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if pyr_type is "octave":
/tmp/ipykernel_63481/614351666.py:15: SyntaxWarning: "is" with a literal. Did you mean "=="?
  elif pyr_type is "halfOctave":
/tmp/ipykernel_63481/614351666.py:18: SyntaxWarning: "is" with a literal. Did you mean "=="?
  elif pyr_type is "smoothHalfOctave":
/tmp/ipykernel_63481/614351666.py:21: SyntaxWarning: "is" with a literal. Did you mean "=="?
  elif pyr_type is "quarterOctave

### Load the resources

In [4]:
video_file = 'IMG_0339_crop.mp4'
video = load_video(video_file)
video.shape

(1659, 960, 702, 3)

In [34]:
magnification_factor = 50
fl = 9
fh = 14
fs = 240
attenuate_other_frequencies=False # False
pyr_type = "halfOctave"
sigma = 5 # 5
temporal_filter = iir_buttler #fir_window_bp


#### Resize our video

In [35]:
# video2 = np.zeros((len(video), 200, 200, 3))
# for i in range(len(video)):
#     video2[i] = sktransform.resize(video[i], (200,200))

# RUN

In [36]:
video_out = phase_amplify(
    video[200:260,:,:,:], 
    magnification_factor, 
    fl, 
    fh, 
    fs, 
    attenuate_other_frequencies=attenuate_other_frequencies, 
    pyramid_type=pyr_type, 
    sigma=sigma, 
    temporal_filter=temporal_filter
)

Using half octave pyramid
Number of filters:,  58
processing level 1
bandpassing...
processing level 2
bandpassing...
processing level 3
bandpassing...
processing level 4
bandpassing...
processing level 5
bandpassing...
processing level 6
bandpassing...
processing level 7
bandpassing...
processing level 8
bandpassing...
processing level 9
bandpassing...
processing level 10
bandpassing...
processing level 11
bandpassing...
processing level 12
bandpassing...
processing level 13
bandpassing...
processing level 14
bandpassing...
processing level 15
bandpassing...
processing level 16
bandpassing...
processing level 17
bandpassing...
processing level 18
bandpassing...
processing level 19
bandpassing...
processing level 20
bandpassing...
processing level 21
bandpassing...
processing level 22
bandpassing...
processing level 23
bandpassing...
processing level 24
bandpassing...
processing level 25
bandpassing...
processing level 26
bandpassing...
processing level 27
bandpassing...
processing lev

In [37]:
fps = 30
video_out_name = 'IMG_0339_out_halfOctave_50x_9-14Hz_5sig_fir_window_bp_grayscale.mp4'
write_video(video_out, fps, video_out_name)

Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
IMAGEIO FFMPEG_WRITER WARNING: input image is not divisible by macro_block_size=16, resizing from (702, 960) to (704, 960) to ensure video compatibility with most codecs and players. To prevent resizing, make your input image divisible by the macro_block_size or set the macro_block_size to 1 (risking incompatibility).
[swscaler @ 0x5c3f040] Warning: data is not aligned! This can lead to a speed loss
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lo

In [18]:
video_out.shape

(5, 960, 702, 3)

In [11]:
video_out

array([[[[0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.],
         ...,
         [0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]],

        [[0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.],
         ...,
         [0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]],

        [[0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.],
         ...,
         [0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]],

        ...,

        [[0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.],
         ...,
         [0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]],

        [[0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.],
         ...,
         [0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]],

        [[0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.],
         ...,
         [0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]]],


       [[[0., 0., 0.],
         [0., 0., 0.],
         [0., 